# Supplementing the Schaefer atlases with Subcortical Structures

in `JointAtlas.ipynb` we downloaded and combined some subcortical atlases.
Now we actually want to combine them with the volumetric Schaefer atlases to get 4S in volume space. 

In [1]:
import os
import nibabel as nb

tfhome = os.getenv("TEMPLATEFLOW_HOME")

nlin6_subcortical = (
    "subcortex_merged/tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-01_dseg.nii.gz"
)
nlin6_subcortical_tsv = (
    "subcortex_merged/tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-01_dseg.tsv"
)
nlin6_subcortical_img = nb.load(nlin6_subcortical)

nlin09c_subcortical = (
    "subcortex_merged/tpl-MNI152NLin2009cAsym_atlas-SubcorticalMerged_res-01_dseg.nii.gz"
)
nlin09c_subcortical_tsv = (
    "subcortex_merged/tpl-MNI152NLin2009cAsym_atlas-SubcorticalMerged_res-01_dseg.tsv"
)
nlin09c_subcortical_img = nb.load(nlin09c_subcortical)

n_subcortical_parcels = 56

In [2]:
from glob import glob
import numpy as np


def check_grids(atlas_files):
    print("checking files:")
    print("\n  " + "\n  ".join(atlas_files))

    print("\ncomparing grids:")
    ref_img = nb.load(atlas_files.pop())

    for try_file in atlas_files:
        try_img = nb.load(try_file)
        if not np.allclose(try_img.affine, ref_img.affine):
            raise Exception(
                "incompatible affines:", try_img, try_img.affine, ref_img.affine
            )

        if not try_img.shape == ref_img.shape:
            raise Exception(
                "incompatible shapes:", try_img, try_img.shape, ref_img.shape
            )

        print(try_img.shape, "==", ref_img.shape)

    print("All grids match orientation and shape!!")


nlin6_schaefers = glob(
    "Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-*_dseg.nii.gz"
)
nlin09c_schaefers = glob(
    "Schaefer/tpl-MNI152NLin2009cAsym_atlas-Schaefer2018v0143_res-01_desc-*_dseg.nii.gz"
)

check_grids([nlin6_subcortical] + nlin6_schaefers)
check_grids([nlin09c_subcortical] + nlin09c_schaefers)

checking files:

  tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-01_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-800ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-500ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-300ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-600ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-700ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-200ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-900ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-400ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-100ParcelsAllNetworks_dseg.nii.gz
  Schaefe

Let's look at what one of these entries looks like:

## Combining the atlases

We need to add the cerebellum, thalamus and subcortical regions into a single atlas. Below are some functions that help us add one atlas into another. We'll add these three together and then add them to all the other atlases that don't have any subcortical regions

This code worked in the other notebooks, no need to test here

### Creating the full volumetric atlas set: NLin6

The Shaefer NLin6 atlases will get the subcortical regions added to them

In [7]:
import pandas as pd

from utils import add_atlas_to_another, roi_data, tsv_to_config

nlin6_subcortical_config = tsv_to_config(nlin6_subcortical_tsv)
nlin6_subcortical_data = roi_data(nlin6_subcortical)

for nlin6_schaefer in nlin6_schaefers:
    resolution = nlin6_schaefer.split("desc-")[1].split("Parcels")[0]
    # Get the image and the tsv
    schaefer_data = roi_data(nlin6_schaefer)
    schaefer_config = tsv_to_config(
        f"Schaefer/atlas-Schaefer2018v0143_desc-{resolution}ParcelsAllNetworks_dseg.tsv"
    )
    combined_config_nlin6, combined_data_nlin6 = add_atlas_to_another(
        schaefer_config,
        schaefer_data,
        nlin6_subcortical_config,
        nlin6_subcortical_data,
    )

    subcortical_nlin6 = nb.Nifti1Image(
        combined_data_nlin6, nlin6_subcortical_img.affine
    )
    new_nparcels = int(resolution) + n_subcortical_parcels
    merged_file_name = f"tpl-MNI152NLin6Asym_atlas-4S{new_nparcels}Parcels_res-01_dseg"
    subcortical_nlin6.to_filename(merged_file_name + ".nii.gz")
    nlin6_df = pd.DataFrame(
        {
            "index": combined_config_nlin6["node_ids"],
            "name": combined_config_nlin6["node_names"],
        },
    )

    # Check this data frame to ensure that the indexes and labels are the same
    official_label_df = pd.read_table(f"atlas-4S{new_nparcels}Parcels_dseg.tsv")
    assert (nlin6_df["index"] == official_label_df["index"]).all()
    assert (nlin6_df["name"] == official_label_df["label"]).all()

PermissionError: [Errno 13] Permission denied: 'tpl-MNI152NLin6Asym_atlas-4S852Parcels_res-01_dseg.nii.gz'

### Creating the full volumetric atlas set: 2009c

The Schaefer 2009c atlases will get the subcortical regions added to them


In [7]:
nlin09c_subcortical_config = tsv_to_config(nlin09c_subcortical_tsv)
nlin09c_subcortical_data = roi_data(nlin09c_subcortical)

for nlin09c_schaefer in nlin09c_schaefers:
    resolution = nlin09c_schaefer.split("desc-")[1].split("Parcels")[0]
    # Get the image and the tsv
    schaefer_data = roi_data(nlin09c_schaefer)
    schaefer_config = tsv_to_config(
        f"Schaefer/atlas-Schaefer2018v0143_desc-{resolution}ParcelsAllNetworks_dseg.tsv"
    )
    combined_config_nlin09c, combined_data_nlin09c = add_atlas_to_another(
        schaefer_config,
        schaefer_data,
        nlin09c_subcortical_config,
        nlin09c_subcortical_data,
    )

    subcortical_nlin09c = nb.Nifti1Image(
        combined_data_nlin09c, nlin09c_subcortical_img.affine
    )
    new_nparcels = int(resolution) + n_subcortical_parcels
    merged_file_name = (
        f"tpl-MNI152NLin2009cAsym_atlas-4S{new_nparcels}Parcels_res-01_dseg"
    )
    subcortical_nlin09c.to_filename(merged_file_name + ".nii.gz")
    nlin09c_df = pd.DataFrame(
        {
            "index": combined_config_nlin09c["node_ids"],
            "name": combined_config_nlin09c["node_names"],
        },
    )

    # Check this data frame to ensure that the indexes and labels are the same
    official_label_df = pd.read_table(f"atlas-4S{new_nparcels}Parcels_dseg.tsv")
    assert (nlin09c_df["index"] == official_label_df["index"]).all()
    assert (nlin09c_df["name"] == official_label_df["label"]).all()